# Pascalov svijet

## Vjerojatnost kao logika neizvjesnosti

U ovom poglavlju istražujemo **teoriju vjerojatnosti** kao prirodno proširenje logike sudova, inspirirani Pascalovim pionirskim radom o igrama na sreću i njegovim filozofskim razmatranjima o neizvjesnosti.

> "Srce ima svoje razloge koje razum ne poznaje" (*Le cœur a ses raisons que la raison ne connaît point*) - Blaise Pascal

Pascal je, zajedno s Fermatom, postavio temelje teorije vjerojatnosti 1654. godine, ali njegov doprinos seže dalje - pokazao je kako matematički pristupiti neizvjesnosti i racionalnom odlučivanju.

## 1. Od logike sudova do vjerojatnosti

U klasičnoj logici, **elementarni sud** može biti samo istinit (⊤) ili lažan (⊥). Vjerojatnost generalizira ovaj koncept dopuštajući **stupnjeve istinitosti** između 0 i 1:

- Klasična logika: $v(\varphi) \in \{\bot, \top\}$
- Vjerojatnost: $P(\varphi) \in [0, 1]$

gdje $P(\varphi) = 0$ odgovara $\bot$, a $P(\varphi) = 1$ odgovara $\top$.

In [12]:
class Sud:
    """Predstavlja sud s klasičnom ili vjerojatnosnom vrijednošću."""
    
    def __init__(self, naziv, klasicna_vrijednost=None, vjerojatnost=None):
        self.naziv = naziv
        self.klasicna = klasicna_vrijednost  # ⊤ ili ⊥
        self.vjerojatnost = vjerojatnost     # [0, 1]
    
    def __repr__(self):
        if self.vjerojatnost is not None:
            return f"P('{self.naziv}') = {self.vjerojatnost:.2f}"
        elif self.klasicna is not None:
            return f"Sud '{self.naziv}': {'⊤' if self.klasicna else '⊥'}"
        return f"Sud '{self.naziv}': neodređen"

# Primjeri sudova
print("Usporedba logike i vjerojatnosti:")
print("="*34)

# Klasična logika
print("\nKlasična logika:")
kisa_klasicno = Sud("Pada kiša", klasicna_vrijednost=True)
sunce_klasicno = Sud("Sunčano je", klasicna_vrijednost=False)
print(f"  {kisa_klasicno}")
print(f"  {sunce_klasicno}")

# Vjerojatnosna logika
print("\nVjerojatnosna logika:")
kisa_vjerojatnost = Sud("Pada kiša", vjerojatnost=0.3)
sunce_vjerojatnost = Sud("Sunčano je", vjerojatnost=0.7)
oblacno_vjerojatnost = Sud("Oblačno", vjerojatnost=0.45)
print(f"  {kisa_vjerojatnost}")
print(f"  {sunce_vjerojatnost}")
print(f"  {oblacno_vjerojatnost}")

# Interpretacija
print("\nInterpretacija:")
interpretacije = [
    (0.0, "⊥ (sigurno lažno)"),
    (0.3, "stupanj vjerovanja 30%"),
    (0.5, "potpuna neizvjesnost"),
    (0.7, "stupanj vjerovanja 70%"),
    (1.0, "⊤ (sigurno istinito)")
]
for p, opis in interpretacije:
    print(f"  P = {p:.2f} ≈ {opis}")

Usporedba logike i vjerojatnosti:

Klasična logika:
  Sud 'Pada kiša': ⊤
  Sud 'Sunčano je': ⊥

Vjerojatnosna logika:
  P('Pada kiša') = 0.30
  P('Sunčano je') = 0.70
  P('Oblačno') = 0.45

Interpretacija:
  P = 0.00 ≈ ⊥ (sigurno lažno)
  P = 0.30 ≈ stupanj vjerovanja 30%
  P = 0.50 ≈ potpuna neizvjesnost
  P = 0.70 ≈ stupanj vjerovanja 70%
  P = 1.00 ≈ ⊤ (sigurno istinito)


## 2. Kolmogorovljevi aksiomi vjerojatnosti

Andrej Kolmogorov je 1933. formalizirao teoriju vjerojatnosti kroz tri elegantna aksioma. Za skup svih mogućih sudova $\Omega$ i vjerojatnosnu mjeru $P$:

1. **Nenegativnost**: $P(\varphi) \geq 0$ za svaki sud $\varphi$
2. **Normalizacija**: $P(\Omega) = 1$ (sigurna istina ima vjerojatnost 1)
3. **Aditivnost**: Za međusobno isključive sudove $\varphi_1, \varphi_2, ...$:
   $$P(\varphi_1 \vee \varphi_2 \vee ...) = P(\varphi_1) + P(\varphi_2) + ...$$

In [13]:
class VjerojatnosniProstor:
    """Predstavlja vjerojatnosni prostor s Kolmogorovljevim aksiomima."""
    
    def __init__(self, omega, vjerojatnosti):
        """
        omega: skup elementarnih sudova
        vjerojatnosti: rječnik {sud: vjerojatnost}
        """
        self.omega = omega
        self.P = vjerojatnosti
        self._provjeri_aksiome()
    
    def _provjeri_aksiome(self):
        """Provjerava zadovoljavaju li vjerojatnosti aksiome."""
        # Aksiom 1: Nenegativnost
        for p in self.P.values():
            if p < 0:
                raise ValueError(f"Vjerojatnost {p} krši aksiom nenegativnosti!")
        
        # Aksiom 2: Normalizacija
        suma = sum(self.P.values())
        if abs(suma - 1.0) > 1e-10:
            raise ValueError(f"Suma vjerojatnosti {suma} ≠ 1 (krši normalizaciju)!")
    
    def vjerojatnost_suda(self, predikat):
        """Računa vjerojatnost složenog suda."""
        p = 0
        for ishod, vjerojatnost in self.P.items():
            if predikat(ishod):
                p += vjerojatnost
        return p

# Primjer: Poštena kocka
kocka_omega = {1, 2, 3, 4, 5, 6}
kocka_P = {i: 1/6 for i in kocka_omega}
kocka = VjerojatnosniProstor(kocka_omega, kocka_P)

print("Demonstracija Kolmogorovljevih aksioma")
print("="*39)
print("\nProstor elementarnih sudova Ω:")
print(f"  Kocka pokazuje: {kocka_omega}")

# Aksiom 1: Nenegativnost
print("\nAKSIOM 1 - Nenegativnost:")
testovi = [
    ("Paran broj", lambda x: x % 2 == 0),
    ("Broj > 4", lambda x: x > 4),
    ("Broj = 7", lambda x: x == 7)
]
for naziv, predikat in testovi:
    p = kocka.vjerojatnost_suda(predikat)
    print(f"  P('{naziv}') = {p:.3f} ≥ 0 ✓")

# Aksiom 2: Normalizacija
print("\nAKSIOM 2 - Normalizacija:")
p_omega = kocka.vjerojatnost_suda(lambda x: True)
print(f"  P(Ω) = P('Bilo koji broj 1-6') = {p_omega:.3f} ✓")

# Aksiom 3: Aditivnost
print("\nAKSIOM 3 - Aditivnost:")
print("  Međusobno isključivi sudovi:")
p1 = kocka.vjerojatnost_suda(lambda x: x == 1)
p3 = kocka.vjerojatnost_suda(lambda x: x == 3)
p5 = kocka.vjerojatnost_suda(lambda x: x == 5)
print(f"    P('Broj = 1') = {p1:.3f}")
print(f"    P('Broj = 3') = {p3:.3f}")
print(f"    P('Broj = 5') = {p5:.3f}")

p_unija = kocka.vjerojatnost_suda(lambda x: x in {1, 3, 5})
p_suma = p1 + p3 + p5

print(f"  \n  P('Broj ∈ {{1,3,5}}') = {p_unija:.3f}")
print(f"  P('1') + P('3') + P('5') = {p_suma:.3f}")
print(f"  Aditivnost vrijedi: {p_unija:.3f} = {p_suma:.3f} ✓")

print("\nPosljedice aksioma:")
print("  P(¬φ) = 1 - P(φ)")
print("  P(∅) = 0")
print("  P(φ ∨ ψ) = P(φ) + P(ψ) - P(φ ∧ ψ)")

Demonstracija Kolmogorovljevih aksioma

Prostor elementarnih sudova Ω:
  Kocka pokazuje: {1, 2, 3, 4, 5, 6}

AKSIOM 1 - Nenegativnost:
  P('Paran broj') = 0.500 ≥ 0 ✓
  P('Broj > 4') = 0.333 ≥ 0 ✓
  P('Broj = 7') = 0.000 ≥ 0 ✓

AKSIOM 2 - Normalizacija:
  P(Ω) = P('Bilo koji broj 1-6') = 1.000 ✓

AKSIOM 3 - Aditivnost:
  Međusobno isključivi sudovi:
    P('Broj = 1') = 0.167
    P('Broj = 3') = 0.167
    P('Broj = 5') = 0.167
  
  P('Broj ∈ {1,3,5}') = 0.500
  P('1') + P('3') + P('5') = 0.500
  Aditivnost vrijedi: 0.500 = 0.500 ✓

Posljedice aksioma:
  P(¬φ) = 1 - P(φ)
  P(∅) = 0
  P(φ ∨ ψ) = P(φ) + P(ψ) - P(φ ∧ ψ)


## 3. Vjerojatnost kao proširenje logičkih veznika

Vjerojatnosni račun generalizira logičke veznike:

- **Negacija**: $P(\neg\varphi) = 1 - P(\varphi)$
- **Disjunkcija**: $P(\varphi \vee \psi) = P(\varphi) + P(\psi) - P(\varphi \wedge \psi)$
- **Konjunkcija**: $P(\varphi \wedge \psi) = P(\varphi) \cdot P(\psi|\varphi)$

Kada su sudovi **nezavisni**: $P(\varphi \wedge \psi) = P(\varphi) \cdot P(\psi)$

In [14]:
class VjerojatnosnaLogika:
    """Implementira vjerojatnosne verzije logičkih veznika."""
    
    @staticmethod
    def negacija(p_phi):
        """P(¬φ) = 1 - P(φ)"""
        return 1 - p_phi
    
    @staticmethod
    def disjunkcija(p_phi, p_psi, p_phi_i_psi):
        """P(φ ∨ ψ) = P(φ) + P(ψ) - P(φ ∧ ψ)"""
        return p_phi + p_psi - p_phi_i_psi
    
    @staticmethod
    def konjunkcija_nezavisna(p_phi, p_psi):
        """P(φ ∧ ψ) = P(φ) · P(ψ) za nezavisne sudove"""
        return p_phi * p_psi
    
    @staticmethod
    def uvjetna_vjerojatnost(p_phi_i_psi, p_psi):
        """P(φ|ψ) = P(φ ∧ ψ) / P(ψ)"""
        if p_psi == 0:
            return None  # Nedefinirana
        return p_phi_i_psi / p_psi
    
    @staticmethod
    def implikacija(p_phi, p_psi, p_phi_i_psi):
        """P(φ → ψ) = P(¬φ ∨ ψ)"""
        p_neg_phi = 1 - p_phi
        # P(¬φ ∧ ψ) = P(ψ) - P(φ ∧ ψ)
        p_neg_phi_i_psi = p_psi - p_phi_i_psi
        return p_neg_phi + p_psi - p_neg_phi_i_psi

# Primjer: Vremenske prilike
vl = VjerojatnosnaLogika()

# Definiraj vjerojatnosti
P_kisa = 0.6      # P(pada kiša)
P_hladno = 0.4    # P(hladno je)
P_kisa_i_hladno = 0.3  # P(pada kiša ∧ hladno je)

print("Vjerojatnosni logički veznici")
print("="*30)
print("\nOsnovni sudovi:")
print(f"  P(A) = {P_kisa:.2f}  ('Pada kiša')")
print(f"  P(B) = {P_hladno:.2f}  ('Hladno je')")
print(f"  P(A ∧ B) = {P_kisa_i_hladno:.2f}  ('Pada kiša i hladno je')")

# Negacija
print("\n1. NEGACIJA:")
P_ne_kisa = vl.negacija(P_kisa)
print(f"  P(¬A) = 1 - P(A) = 1 - {P_kisa:.2f} = {P_ne_kisa:.2f}")
print(f"  Interpretacija: Vjerojatnost da ne pada kiša je {P_ne_kisa*100:.0f}%")

# Disjunkcija
print("\n2. DISJUNKCIJA:")
P_kisa_ili_hladno = vl.disjunkcija(P_kisa, P_hladno, P_kisa_i_hladno)
print(f"  P(A ∨ B) = P(A) + P(B) - P(A ∧ B)")
print(f"  P(A ∨ B) = {P_kisa:.2f} + {P_hladno:.2f} - {P_kisa_i_hladno:.2f} = {P_kisa_ili_hladno:.2f}")
print(f"  Interpretacija: Vjerojatnost da pada kiša ili je hladno je {P_kisa_ili_hladno*100:.0f}%")

# Implikacija
print("\n3. IMPLIKACIJA:")
P_ako_kisa_onda_hladno = vl.implikacija(P_kisa, P_hladno, P_kisa_i_hladno)
print(f"  P(A → B) = P(¬A ∨ B) = P(¬A) + P(B) - P(¬A ∧ B)")
P_ne_kisa_i_hladno = P_hladno - P_kisa_i_hladno
print(f"  P(A → B) = {P_ne_kisa:.2f} + {P_hladno:.2f} - {P_ne_kisa_i_hladno:.2f} = {P_ako_kisa_onda_hladno:.2f}")
print(f"  Interpretacija: Vjerojatnost da 'ako pada kiša, onda je hladno' je {P_ako_kisa_onda_hladno*100:.0f}%")

# Uvjetna vjerojatnost
print("\n4. UVJETNA VJEROJATNOST:")
P_hladno_ako_kisa = vl.uvjetna_vjerojatnost(P_kisa_i_hladno, P_kisa)
print(f"  P(B|A) = P(A ∧ B) / P(A) = {P_kisa_i_hladno:.2f} / {P_kisa:.2f} = {P_hladno_ako_kisa:.2f}")
print(f"  Interpretacija: Ako pada kiša, vjerojatnost da je hladno je {P_hladno_ako_kisa*100:.0f}%")

# Test nezavisnosti
print("\nTest nezavisnosti:")
P_nezavisno = vl.konjunkcija_nezavisna(P_kisa, P_hladno)
print(f"  P(A) · P(B) = {P_kisa:.2f} · {P_hladno:.2f} = {P_nezavisno:.2f}")
print(f"  P(A ∧ B) = {P_kisa_i_hladno:.2f}")
if abs(P_nezavisno - P_kisa_i_hladno) < 0.01:
    print(f"  Budući da {P_nezavisno:.2f} ≈ {P_kisa_i_hladno:.2f}, sudovi su približno nezavisni")
else:
    print(f"  Budući da {P_nezavisno:.2f} ≠ {P_kisa_i_hladno:.2f}, sudovi NISU nezavisni")
    print("  (Kiša i hladnoća su povezani!)")

Vjerojatnosni logički veznici

Osnovni sudovi:
  P(A) = 0.60  ('Pada kiša')
  P(B) = 0.40  ('Hladno je')
  P(A ∧ B) = 0.30  ('Pada kiša i hladno je')

1. NEGACIJA:
  P(¬A) = 1 - P(A) = 1 - 0.60 = 0.40
  Interpretacija: Vjerojatnost da ne pada kiša je 40%

2. DISJUNKCIJA:
  P(A ∨ B) = P(A) + P(B) - P(A ∧ B)
  P(A ∨ B) = 0.60 + 0.40 - 0.30 = 0.70
  Interpretacija: Vjerojatnost da pada kiša ili je hladno je 70%

3. IMPLIKACIJA:
  P(A → B) = P(¬A ∨ B) = P(¬A) + P(B) - P(¬A ∧ B)
  P(A → B) = 0.40 + 0.40 - 0.10 = 0.70
  Interpretacija: Vjerojatnost da 'ako pada kiša, onda je hladno' je 70%

4. UVJETNA VJEROJATNOST:
  P(B|A) = P(A ∧ B) / P(A) = 0.30 / 0.60 = 0.50
  Interpretacija: Ako pada kiša, vjerojatnost da je hladno je 50%

Test nezavisnosti:
  P(A) · P(B) = 0.60 · 0.40 = 0.24
  P(A ∧ B) = 0.30
  Budući da 0.24 ≠ 0.30, sudovi NISU nezavisni
  (Kiša i hladnoća su povezani!)


## Bayesov teorem - zaključivanje s neizvjesnošću

**Bayesov teorem** omogućava ažuriranje vjerovanja na temelju novih dokaza:

> "Vjerojatnost je vodič života" - Ciceron, anticipirajući Bayesovu logiku

$$P(H|E) = \frac{P(E|H) \cdot P(H)}{P(E)}$$

gdje je:
- $P(H)$ - **apriorna vjerojatnost** hipoteze
- $P(E|H)$ - **izglednost** dokaza ako je hipoteza istinita
- $P(H|E)$ - **aposteriorna vjerojatnost** nakon dokaza



In [15]:
class BayesovoZakljucivanje:
    """Implementira Bayesovo ažuriranje vjerojatnosti."""
    
    def __init__(self, apriorna, izglednosti):
        """
        apriorna: dict {hipoteza: P(hipoteza)}
        izglednosti: dict {hipoteza: {dokaz: P(dokaz|hipoteza)}}
        """
        self.apriorna = apriorna
        self.izglednosti = izglednosti
    
    def azuriraj(self, dokaz):
        """Ažurira vjerojatnosti na temelju novog dokaza."""
        # Izračunaj P(dokaz)
        p_dokaz = 0
        for hipoteza, p_h in self.apriorna.items():
            p_dokaz += self.izglednosti[hipoteza][dokaz] * p_h
        
        # Bayesovo ažuriranje za svaku hipotezu
        aposteriorna = {}
        for hipoteza, p_h in self.apriorna.items():
            p_dokaz_ako_h = self.izglednosti[hipoteza][dokaz]
            aposteriorna[hipoteza] = (p_dokaz_ako_h * p_h) / p_dokaz
        
        return aposteriorna, p_dokaz

# Primjer: Medicinska dijagnoza
print("Bayesovo zaključivanje - Medicinska dijagnoza")
print("="*46)
print("\nScenarij: Test za rijetku bolest\n")

# Definiraj vjerojatnosti
apriorna = {
    "bolest": 0.001,     # 1 od 1000 ljudi ima bolest
    "zdrav": 0.999
}

izglednosti = {
    "bolest": {
        "pozitivan": 0.99,  # Osjetljivost testa (true positive rate)
        "negativan": 0.01   # False negative rate
    },
    "zdrav": {
        "pozitivan": 0.05,  # False positive rate
        "negativan": 0.95   # True negative rate
    }
}

bayes = BayesovoZakljucivanje(apriorna, izglednosti)

print("Početne informacije:")
print(f"  P(Bolest) = {apriorna['bolest']:.3f} (1 od 1000 ljudi ima bolest)")
print(f"  P(Test+|Bolest) = {izglednosti['bolest']['pozitivan']:.3f} (osjetljivost testa)")
print(f"  P(Test+|¬Bolest) = {izglednosti['zdrav']['pozitivan']:.3f} (lažno pozitivna stopa)")

print("\nPitanje: Ako je test pozitivan, kolika je vjerojatnost bolesti?")

# Bayesovo ažuriranje
aposteriorna, p_pozitivan = bayes.azuriraj("pozitivan")

print("\nBayesov račun:")
print("1. P(Test+) = P(Test+|Bolest)·P(Bolest) + P(Test+|¬Bolest)·P(¬Bolest)")
p1 = izglednosti['bolest']['pozitivan'] * apriorna['bolest']
p2 = izglednosti['zdrav']['pozitivan'] * apriorna['zdrav']
print(f"   P(Test+) = {izglednosti['bolest']['pozitivan']:.3f} · {apriorna['bolest']:.3f} + "
      f"{izglednosti['zdrav']['pozitivan']:.3f} · {apriorna['zdrav']:.3f}")
print(f"   P(Test+) = {p1:.5f} + {p2:.5f} = {p_pozitivan:.5f}")

print("\n2. P(Bolest|Test+) = P(Test+|Bolest) · P(Bolest) / P(Test+)")
print(f"   P(Bolest|Test+) = {izglednosti['bolest']['pozitivan']:.3f} · "
      f"{apriorna['bolest']:.3f} / {p_pozitivan:.5f}")
print(f"   P(Bolest|Test+) = {aposteriorna['bolest']:.5f}")

print("\nZAKLJUČAK:")
print(f"  Apriorna vjerojatnost: {apriorna['bolest']*100:.1f}%")
print(f"  Aposteriorna vjerojatnost (nakon pozitivnog testa): {aposteriorna['bolest']*100:.1f}%")
print("  \n  Iako je test vrlo pouzdan (99% osjetljivost),")
print(f"  pozitivan test povećava vjerojatnost bolesti samo na {aposteriorna['bolest']*100:.1f}%!")
print("  \n  Razlog: Bolest je vrlo rijetka, pa većina pozitivnih")
print("  testova su lažno pozitivni.")

# Vizualizacija
print("\nVizualizacija od 100,000 ljudi:")
n_ljudi = 100000
n_bolesnih = int(n_ljudi * apriorna['bolest'])
n_zdravih = n_ljudi - n_bolesnih

tp = int(n_bolesnih * izglednosti['bolest']['pozitivan'])  # True positive
fn = n_bolesnih - tp  # False negative
fp = int(n_zdravih * izglednosti['zdrav']['pozitivan'])  # False positive
tn = n_zdravih - fp  # True negative

print(f"  Bolesni: {n_bolesnih} ljudi")
print(f"    → Pozitivan test: {tp} (istinito pozitivni)")
print(f"    → Negativan test: {fn} (lažno negativan)")
print(f"  \n  Zdravi: {n_zdravih:,} ljudi")
print(f"    → Pozitivan test: {fp:,} (lažno pozitivni)")
print(f"    → Negativan test: {tn:,} (istinito negativni)")
print(f"  \n  Ukupno pozitivnih testova: {tp + fp:,}")
print(f"  Od toga stvarno bolesnih: {tp}")
print(f"  Vjerojatnost bolesti kod pozitivnog testa: {tp}/{tp+fp} = {tp/(tp+fp)*100:.1f}%")

Bayesovo zaključivanje - Medicinska dijagnoza

Scenarij: Test za rijetku bolest

Početne informacije:
  P(Bolest) = 0.001 (1 od 1000 ljudi ima bolest)
  P(Test+|Bolest) = 0.990 (osjetljivost testa)
  P(Test+|¬Bolest) = 0.050 (lažno pozitivna stopa)

Pitanje: Ako je test pozitivan, kolika je vjerojatnost bolesti?

Bayesov račun:
1. P(Test+) = P(Test+|Bolest)·P(Bolest) + P(Test+|¬Bolest)·P(¬Bolest)
   P(Test+) = 0.990 · 0.001 + 0.050 · 0.999
   P(Test+) = 0.00099 + 0.04995 = 0.05094

2. P(Bolest|Test+) = P(Test+|Bolest) · P(Bolest) / P(Test+)
   P(Bolest|Test+) = 0.990 · 0.001 / 0.05094
   P(Bolest|Test+) = 0.01943

ZAKLJUČAK:
  Apriorna vjerojatnost: 0.1%
  Aposteriorna vjerojatnost (nakon pozitivnog testa): 1.9%
  
  Iako je test vrlo pouzdan (99% osjetljivost),
  pozitivan test povećava vjerojatnost bolesti samo na 1.9%!
  
  Razlog: Bolest je vrlo rijetka, pa većina pozitivnih
  testova su lažno pozitivni.

Vizualizacija od 100,000 ljudi:
  Bolesni: 100 ljudi
    → Pozitivan test: 99

## 5. Pascalova oklada - filozofija vjerojatnosti

Pascal je primijenio vjerojatnosno razmišljanje na ultimativno filozofsko pitanje - postojanje Boga:

> "Morate se kladiti. To nije dobrovoljno, primorani ste na to" - Pascal, *Pensées*

Pascalova matrica odlučivanja:

| | Bog postoji | Bog ne postoji |
|---|---|---|
| **Vjerujem** | +∞ (vječna sreća) | -c (mali gubitak) |
| **Ne vjerujem** | -∞ (vječna kazna) | +c (mali dobitak) |

Čak i ako je $P(\text{Bog postoji}) = \epsilon$ vrlo mala, očekivana korisnost vjerovanja je beskonačna!

In [16]:
import math

def fmt_prob(x: float) -> str:
    return f"{x:.2f}" if x >= 0.01 else f"{x:.4f}"

class PascalovaOklada:
    """Simulacija Pascalove oklade kroz teoriju odlučivanja."""
    
    def __init__(self):
        # Matrica korisnosti (payoff matrix)
        # Koristimo velike brojeve umjesto beskonačnosti za demonstraciju
        self.korisnost = {
            ("vjeruj", "postoji"): float('inf'),    # Vječna sreća
            ("vjeruj", "ne_postoji"): -10,          # Mali gubitak (trud vjerovanja)
            ("ne_vjeruj", "postoji"): float('-inf'), # Vječna kazna
            ("ne_vjeruj", "ne_postoji"): 10         # Mali dobitak (sloboda)
        }
    
    def ocekivana_korisnost(self, akcija, p_bog):
        """Računa očekivanu korisnost akcije."""
        k_postoji = self.korisnost[(akcija, "postoji")]
        k_ne_postoji = self.korisnost[(akcija, "ne_postoji")]
        
        return p_bog * k_postoji + (1 - p_bog) * k_ne_postoji
    
    def optimalna_odluka(self, p_bog):
        """Vraća optimalnu odluku za danu vjerojatnost."""
        eu_vjeruj = self.ocekivana_korisnost("vjeruj", p_bog)
        eu_ne_vjeruj = self.ocekivana_korisnost("ne_vjeruj", p_bog)
        
        if eu_vjeruj > eu_ne_vjeruj:
            return "vjeruj", eu_vjeruj, eu_ne_vjeruj
        elif eu_ne_vjeruj > eu_vjeruj:
            return "ne_vjeruj", eu_vjeruj, eu_ne_vjeruj
        else:
            return "svejedno", eu_vjeruj, eu_ne_vjeruj

# Demonstracija
oklada = PascalovaOklada()

print("Pascalova oklada - Analiza odlučivanja")
print("="*39)
print("\nMatrica korisnosti:")
print("                 Bog postoji    Bog ne postoji")
print("  Vjerujem:      +∞             -10           ")
print("  Ne vjerujem:   -∞             +10           ")

print("\nAnaliza za različite vjerojatnosti postojanja Boga:\n")

vjerojatnosti = [0.5, 0.1, 0.01, 0.0001]
for p in vjerojatnosti:
    odluka, eu_v, eu_nv = oklada.optimalna_odluka(p)
    
    # Example usage (replace your print block with this structure)
    eu_v_str = f"{eu_v:.2f}" if (eu_v not in (float('inf'), float('-inf'))) else ("∞" if eu_v > 0 else "-∞")
    eu_nv_str = f"{eu_nv:.2f}" if (eu_nv not in (float('inf'), float('-inf'))) else ("∞" if eu_nv > 0 else "-∞")

    p_str = fmt_prob(p)
    one_minus_p_str = fmt_prob(1 - p)

    print(f"  E[vjerujem] = {p_str} · ∞ + {one_minus_p_str} · (-10) = {eu_v_str}")
    print(f"  E[ne vjerujem] = {p_str} · (-∞) + {one_minus_p_str} · 10 = {eu_nv_str}")
    print(f"  → Racionalna odluka: {'VJERUJ' if odluka == 'vjeruj' else 'NE VJERUJ'}\n")

print("Pascalov zaključak:")
print("  Čak i uz infinitezimalnu vjerojatnost postojanja Boga,")
print("  racionalno je vjerovati zbog beskonačne nagrade/kazne.")

print("\nFilozofske kritike:")
kritike = [
    "Problem mnoštva religija (koja vjera?)",
    "Autentičnost vjerovanja (može li se 'odlučiti' vjerovati?)",
    "Beskonačnost kao korisnost (ima li smisla?)",
    "Moralni prigovor (je li to pravo vjerovanje?)"
]
for i, kritika in enumerate(kritike, 1):
    print(f"  {i}. {kritika}")

# Konačna verzija
print("\nKonačna aproksimacija (bez beskonačnosti):")
print("  Korisnosti: vjeruj(Bog da)=1000, vjeruj(Bog ne)=-10")
print("             ne vjeruj(Bog da)=-1000, ne vjeruj(Bog ne)=10")

# Kritična vjerojatnost
# 1000p - 10(1-p) = -1000p + 10(1-p)
# 1000p - 10 + 10p = -1000p + 10 - 10p
# 2020p = 20
p_kritična = 20 / 2020
print(f"  \n  Kritična vjerojatnost: P* = {p_kritična:.3f}")
print(f"  Ako P(Bog) > {p_kritična:.3f}, vjeruj; inače ne vjeruj.")

Pascalova oklada - Analiza odlučivanja

Matrica korisnosti:
                 Bog postoji    Bog ne postoji
  Vjerujem:      +∞             -10           
  Ne vjerujem:   -∞             +10           

Analiza za različite vjerojatnosti postojanja Boga:

  E[vjerujem] = 0.50 · ∞ + 0.50 · (-10) = ∞
  E[ne vjerujem] = 0.50 · (-∞) + 0.50 · 10 = -∞
  → Racionalna odluka: VJERUJ

  E[vjerujem] = 0.10 · ∞ + 0.90 · (-10) = ∞
  E[ne vjerujem] = 0.10 · (-∞) + 0.90 · 10 = -∞
  → Racionalna odluka: VJERUJ

  E[vjerujem] = 0.01 · ∞ + 0.99 · (-10) = ∞
  E[ne vjerujem] = 0.01 · (-∞) + 0.99 · 10 = -∞
  → Racionalna odluka: VJERUJ

  E[vjerujem] = 0.0001 · ∞ + 1.00 · (-10) = ∞
  E[ne vjerujem] = 0.0001 · (-∞) + 1.00 · 10 = -∞
  → Racionalna odluka: VJERUJ

Pascalov zaključak:
  Čak i uz infinitezimalnu vjerojatnost postojanja Boga,
  racionalno je vjerovati zbog beskonačne nagrade/kazne.

Filozofske kritike:
  1. Problem mnoštva religija (koja vjera?)
  2. Autentičnost vjerovanja (može li se 'odlučiti

## 6. Problem indukcije i vjerojatnost

David Hume je ukazao na **problem indukcije** - nemožnost logičkog opravdanja generalizacije iz konačnog broja opažanja. Vjerojatnost nudi djelomično rješenje:

> "Navika je veliki vodič ljudskog života" - Hume

Umjesto traženja sigurnosti, vjerojatnost kvantificira stupanj racionalnog vjerovanja.

In [17]:
class InduktivnoZakljucivanje:
    """Modelira induktivno zaključivanje kroz vjerojatnost."""
    
    def __init__(self):
        self.povijest = []
    
    def laplace_sukcesija(self, uspjesi, pokusaji):
        """Laplace-ov zakon sukcesije.
        
        P(sljedeći uspjeh) = (k + 1) / (n + 2)
        gdje je k broj uspjeha, n broj pokušaja
        """
        return (uspjesi + 1) / (pokusaji + 2)
    
    def azuriraj_vjerovanje(self, novi_dokaz):
        """Ažurira vjerovanje na temelju novog dokaza."""
        self.povijest.append(novi_dokaz)
        uspjesi = sum(self.povijest)
        pokusaji = len(self.povijest)
        return self.laplace_sukcesija(uspjesi, pokusaji)

# Primjer: Sunce izlazi svaki dan
print("Problem indukcije - Sunce izlazi")
print("="*33)

indukcija = InduktivnoZakljucivanje()

print("\nLaplace-ov zakon sukcesije:")
print("  P(uspjeh) = (k + 1) / (n + 2)")
print("  gdje je k = broj dosadašnjih uspjeha, n = ukupan broj pokušaja")

print("\nDana\tP(sunce izađe)\tPromjena")
print("-"*4 + "\t" + "-"*14 + "\t" + "-"*8)

# Simuliraj promatranje sunca
dani = [0, 1, 2, 3, 4, 5, 10, 30, 100, 365, 1000, 10000]
prethodna_p = 0.5

for dan in dani:
    p = indukcija.laplace_sukcesija(dan, dan)
    promjena = p - prethodna_p if dan > 0 else 0
    
    print(f"{dan}\t{p:.4f}\t\t", end="")
    if dan > 0:
        print(f"+{promjena:.4f}" if promjena > 0 else f"{promjena:.4f}")
    else:
        print("-")
    
    if dan in [1, 5, 30, 365, 10000]:
        prethodna_p = p

print("\nHumeov problem:")
print("  Koliko god puta sunce izašlo, P(sunce izađe sutra) < 1")
print("  Nikad ne možemo biti potpuno sigurni!")

# Crni labud
print("\nCrni labud scenarij:")
bijeli_labudovi = 1000
p_bijeli = indukcija.laplace_sukcesija(bijeli_labudovi, bijeli_labudovi)
print(f"Nakon {bijeli_labudovi} bijelih labudova, P(sljedeći bijel) = {p_bijeli:.4f}")
print("Ali jedan crni labud mijenja sve!")

# Nakon crnog labuda
ukupno = bijeli_labudovi + 1
p_bijeli_novi = indukcija.laplace_sukcesija(bijeli_labudovi, ukupno)
p_crni = indukcija.laplace_sukcesija(1, ukupno)

print("\nNakon crnog labuda:")
print(f"  Vidjeli: {bijeli_labudovi} bijelih, 1 crni")
print(f"  P(sljedeći bijel) = {p_bijeli_novi:.4f}")
print(f"  P(sljedeći crni) = {p_crni:.4f}")

print("\nFilozofska pouka:")
print("  Indukcija ne daje sigurnost, već racionalne stupnjeve vjerovanja.")
print("  Vjerojatnost kvantificira našu neizvjesnost o budućnosti.")

Problem indukcije - Sunce izlazi

Laplace-ov zakon sukcesije:
  P(uspjeh) = (k + 1) / (n + 2)
  gdje je k = broj dosadašnjih uspjeha, n = ukupan broj pokušaja

Dana	P(sunce izađe)	Promjena
----	--------------	--------
0	0.5000		-
1	0.6667		+0.1667
2	0.7500		+0.0833
3	0.8000		+0.1333
4	0.8333		+0.1667
5	0.8571		+0.1905
10	0.9167		+0.0595
30	0.9688		+0.1116
100	0.9902		+0.0214
365	0.9973		+0.0285
1000	0.9990		+0.0017
10000	0.9999		+0.0026

Humeov problem:
  Koliko god puta sunce izašlo, P(sunce izađe sutra) < 1
  Nikad ne možemo biti potpuno sigurni!

Crni labud scenarij:
Nakon 1000 bijelih labudova, P(sljedeći bijel) = 0.9990
Ali jedan crni labud mijenja sve!

Nakon crnog labuda:
  Vidjeli: 1000 bijelih, 1 crni
  P(sljedeći bijel) = 0.9980
  P(sljedeći crni) = 0.0020

Filozofska pouka:
  Indukcija ne daje sigurnost, već racionalne stupnjeve vjerovanja.
  Vjerojatnost kvantificira našu neizvjesnost o budućnosti.


## 7. Monty Hall problem - paradoksi uvjetne vjerojatnosti

Monty Hall problem ilustrira kako naša intuicija često griješi kod uvjetnih vjerojatnosti:

In [18]:
import random

class MontyHall:
    """Simulacija Monty Hall problema."""
    
    def igraj(self, strategija="promijeni"):
        """Igra jednu rundu Monty Hall igre.
        
        strategija: 'ostani' ili 'promijeni'
        """
        # Postavi igru
        vrata = [1, 2, 3]
        auto = random.choice(vrata)
        
        # Igrač bira
        izbor = random.choice(vrata)
        
        # Voditelj otvara vrata s kozom
        moguca_vrata = [v for v in vrata if v != izbor and v != auto]
        voditelj_otvara = random.choice(moguca_vrata) if moguca_vrata else \
                          [v for v in vrata if v != izbor][0]
        
        # Primijeni strategiju
        if strategija == "promijeni":
            preostala = [v for v in vrata if v != izbor and v != voditelj_otvara]
            konacni_izbor = preostala[0]
        else:  # ostani
            konacni_izbor = izbor
        
        return konacni_izbor == auto
    
    def simuliraj(self, n_igara=10000):
        """Simulira više igara s obje strategije."""
        rezultati = {
            "ostani": 0,
            "promijeni": 0
        }
        
        for _ in range(n_igara):
            if self.igraj("ostani"):
                rezultati["ostani"] += 1
            if self.igraj("promijeni"):
                rezultati["promijeni"] += 1
        
        return rezultati, n_igara

# Simulacija
mh = MontyHall()
rezultati, n = mh.simuliraj(10000)

print("Monty Hall Problem - Simulacija")
print("="*32)
print("\nPostavka:")
print("  3 vrata: iza jednih je automobil, iza drugih dvoje koze")
print("  1. Birate vrata")
print("  2. Voditelj otvara druga vrata s kozom")
print("  3. Možete promijeniti izbor ili ostati")

print(f"\nSimulacija {n:,} igara:\n")

for strategija, pobjede in rezultati.items():
    postotak = (pobjede / n) * 100
    print(f"Strategija {strategija.upper()}:")
    print(f"  Pobjede: {pobjede:,} / {n:,}")
    print(f"  Postotak pobjede: {postotak:.1f}%\n")

# Bayesova analiza
print("Bayesova analiza:")
print("="*18)
print("\nPočetne vjerojatnosti:")
print("  P(auto na vratima 1) = 1/3")
print("  P(auto na vratima 2) = 1/3")
print("  P(auto na vratima 3) = 1/3")

print("\nBirate vrata 1. Voditelj otvara vrata 3 (koza).")

print("\nAžurirane vjerojatnosti:")
print("  P(auto na 1 | voditelj otvorio 3) = 1/3")
print("  P(auto na 2 | voditelj otvorio 3) = 2/3")
print("  P(auto na 3 | voditelj otvorio 3) = 0")

print("\nObjašnjenje:")
print("  Voditelj ZNA gdje je auto i MORA otvoriti kozu.")
print("  Njegovo otvaranje daje informaciju!")
print("  ")
print("  Ako je auto na vratima 1: voditelj bira između 2 i 3 (P=1/2)")
print("  Ako je auto na vratima 2: voditelj MORA otvoriti 3 (P=1)")
print("  Ako je auto na vratima 3: voditelj MORA otvoriti 2 (P=1)")
print("  ")
print("  Činjenica da je otvorio 3 favorizira vrata 2!")

print("\nZaključak: UVIJEK se isplati promijeniti izbor!")

Monty Hall Problem - Simulacija

Postavka:
  3 vrata: iza jednih je automobil, iza drugih dvoje koze
  1. Birate vrata
  2. Voditelj otvara druga vrata s kozom
  3. Možete promijeniti izbor ili ostati

Simulacija 10,000 igara:

Strategija OSTANI:
  Pobjede: 3,219 / 10,000
  Postotak pobjede: 32.2%

Strategija PROMIJENI:
  Pobjede: 6,712 / 10,000
  Postotak pobjede: 67.1%

Bayesova analiza:

Početne vjerojatnosti:
  P(auto na vratima 1) = 1/3
  P(auto na vratima 2) = 1/3
  P(auto na vratima 3) = 1/3

Birate vrata 1. Voditelj otvara vrata 3 (koza).

Ažurirane vjerojatnosti:
  P(auto na 1 | voditelj otvorio 3) = 1/3
  P(auto na 2 | voditelj otvorio 3) = 2/3
  P(auto na 3 | voditelj otvorio 3) = 0

Objašnjenje:
  Voditelj ZNA gdje je auto i MORA otvoriti kozu.
  Njegovo otvaranje daje informaciju!
  
  Ako je auto na vratima 1: voditelj bira između 2 i 3 (P=1/2)
  Ako je auto na vratima 2: voditelj MORA otvoriti 3 (P=1)
  Ako je auto na vratima 3: voditelj MORA otvoriti 2 (P=1)
  
  Činjen

## 8. Filozofske interpretacije vjerojatnosti

Postoje različite filozofske interpretacije što vjerojatnost zapravo znači:

1. **Klasična** (Laplace): Omjer povoljnih i mogućih ishoda
2. **Frekventistička** (von Mises): Granična relativna frekvencija
3. **Propenzitetna** (Popper): Objektivna tendencija
4. **Subjektivna/Bayesova** (de Finetti): Stupanj racionalnog vjerovanja

Svaka interpretacija ima filozofske implikacije za prirodu slučajnosti, determinizma i znanja.

In [19]:
def filozofske_interpretacije():
    """Demonstrira različite filozofske interpretacije vjerojatnosti."""
    
    print("Filozofske interpretacije vjerojatnosti")
    print("="*40)
    
    # 1. Klasična
    print("\n1. KLASIČNA INTERPRETACIJA (Laplace)")
    print("   'Vjerojatnost je omjer povoljnih i jednakovjerojatnih ishoda'")
    print("   \n   Primjer: Kocka")
    povoljni = 3  # parni brojevi: 2, 4, 6
    mogući = 6
    p_klasična = povoljni / mogući
    print(f"   P(paran broj) = {povoljni}/{mogući} = {p_klasična:.3f}")
    print("   \n   Problem: Pretpostavlja jednakovjerojatnost (cirkularnost)")
    
    # 2. Frekventistička
    print("\n2. FREKVENTISTIČKA INTERPRETACIJA (von Mises)")
    print("   'Vjerojatnost je granična relativna frekvencija'")
    print("   ")
    
    random.seed(42)
    for n in [100, 1000, 10000, 100000]:
        parni = sum(1 for _ in range(n) if random.randint(1, 6) % 2 == 0)
        frekvencija = parni / n
        print(f"   Bacanja: {n}, Parni: {parni}, Frekvencija: {frekvencija:.3f}")
    print("   → Konvergira prema 0.500")
    print("   \n   Problem: Što s jedinstvenim događajima?")
    
    # 3. Propenzitetna
    print("\n3. PROPENZITETNA INTERPRETACIJA (Popper)")
    print("   'Vjerojatnost je objektivna tendencija sustava'")
    print("   ")
    print("   Kocka ima propenzitet 0.5 za paran broj")
    print("   zbog svoje fizičke strukture.")
    print("   \n   Problem: Kako mjeriti propenzitet?")
    
    # 4. Subjektivna
    print("\n4. SUBJEKTIVNA INTERPRETACIJA (de Finetti)")
    print("   'Vjerojatnost je stupanj racionalnog vjerovanja'")
    print("   ")
    print("   Moje vjerovanje:")
    print("   P(kiša sutra) = 0.30 (na temelju prognoze)")
    print("   ")
    print("   Koherentnost: Moja vjerovanja moraju zadovoljiti")
    print("   aksiome vjerojatnosti inače mogu biti 'Dutch booked'")
    print("   \n   Problem: Subjektivnost vs. objektivnost")
    
    # Filozofske implikacije
    print("\nFilozofske implikacije:")
    print("="*24)
    
    print("\nDETERMINIZAM vs. INDETERMINIZAM:")
    print("  - Je li vjerojatnost samo naša neznanja (epistemička)?")
    print("  - Ili je svijet inherentno slučajan (ontološka)?")
    
    print("\nPROBLEM REFERENTNE KLASE:")
    print("  - Jesam li '35-godišnjak', 'Hrvat', 'filozof'?")
    print("  - Različite klase daju različite vjerojatnosti!")
    
    print("\nPRINCIP INDIFERENCIJE:")
    print("  - Bez informacija, dodjeli jednake vjerojatnosti?")
    print("  - Bertrandov paradoks pokazuje probleme")
    
    print("\n> \"Bog ne igra kocke\" - Einstein")
    print("> \"Einstein, prestani govoriti Bogu što da radi\" - Bohr")

filozofske_interpretacije()

Filozofske interpretacije vjerojatnosti

1. KLASIČNA INTERPRETACIJA (Laplace)
   'Vjerojatnost je omjer povoljnih i jednakovjerojatnih ishoda'
   
   Primjer: Kocka
   P(paran broj) = 3/6 = 0.500
   
   Problem: Pretpostavlja jednakovjerojatnost (cirkularnost)

2. FREKVENTISTIČKA INTERPRETACIJA (von Mises)
   'Vjerojatnost je granična relativna frekvencija'
   
   Bacanja: 100, Parni: 49, Frekvencija: 0.490
   Bacanja: 1000, Parni: 510, Frekvencija: 0.510
   Bacanja: 10000, Parni: 5057, Frekvencija: 0.506
   Bacanja: 100000, Parni: 50215, Frekvencija: 0.502
   → Konvergira prema 0.500
   
   Problem: Što s jedinstvenim događajima?

3. PROPENZITETNA INTERPRETACIJA (Popper)
   'Vjerojatnost je objektivna tendencija sustava'
   
   Kocka ima propenzitet 0.5 za paran broj
   zbog svoje fizičke strukture.
   
   Problem: Kako mjeriti propenzitet?

4. SUBJEKTIVNA INTERPRETACIJA (de Finetti)
   'Vjerojatnost je stupanj racionalnog vjerovanja'
   
   Moje vjerovanje:
   P(kiša sutra) = 0.30 (n

## 9. Prijedlozi za daljnje istraživanje

Za produbljivanje razumijevanja vjerojatnosti kao proširenja logike kroz praktične Python zadatke:

### 1. **Fuzzy logika**
Implementirajte fuzzy logičke operatore gdje istinite vrijednosti mogu biti bilo koji broj između 0 i 1. Definirajte Łukasiewicz, Gödel i product t-norme. Pokažite kako se klasična logika dobiva kao granični slučaj.

### 2. **Cox-Jaynesovi teoremi**
Dokažite da svaki sustav koji zadovoljava razumne uvjete za stupnjeve vjerovanja mora biti izomorfan s teorijom vjerojatnosti. Implementirajte alternativni sustav i pokažite kako se svodi na vjerojatnost.

### 3. **Dutch book argument**
Simulirajte klađenje gdje agent s nekoherentnim vjerovanjima (koja krše aksiome vjerojatnosti) uvijek gubi novac. Vizualizirajte kako koherentnost štiti od sigurnog gubitka.

### 4. **Dempster-Shafer teorija**
Implementirajte teoriju dokaza koja generalizira vjerojatnost dopuštajući "ne znam" odgovore. Pokažite kombiniranje dokaza i rad s intervalima vjerojatnosti.

### 5. **Paradoks spavajuće ljepotice**
Simulirajte ovaj paradoks samolociranja u vremenu. Analizirajte sukob između "halfer" i "thirder" pozicija. Povežite s antropijskim principom.

### 6. **Kvantna vjerojatnost**
Implementirajte Born rule i pokažite kako kvantne amplitude daju vjerojatnosti. Demonstrirajte narušavanje Bell-ovih nejednakosti klasičnom vjerojatnošću.

### 7. **Algoritamska vjerojatnost**
Implementirajte Solomonoff-ovu indukciju koristeći Kolmogorovljevu složenost. Pokažite kako kraći programi imaju veću apriornu vjerojatnost.

### 8. **Kauzalno zaključivanje**
Implementirajte Pearl-ove kauzalne grafove. Pokažite razliku između korelacije i kauzalnosti. Simulirajte Simpson-ov paradoks.

### 9. **Maksimalna entropija**
Implementirajte princip maksimalne entropije za izbor vjerojatnosnih distribucija. Pokažite kako ograničenja određuju optimalnu distribuciju.

### 10. **Vjerojatnosno programiranje**
Stvorite jednostavan vjerojatnosni programski jezik gdje varijable mogu biti distribucije. Implementirajte inferenciju kroz MCMC ili variational Bayes.

Svaki zadatak postupno gradi razumijevanje vjerojatnosti kao generalizacije logike, omogućavajući studentima da istraže dublje veze između logike, vjerojatnosti i racionalnog zaključivanja.

## Zaključak

Kroz ovu implementaciju istražili smo teoriju vjerojatnosti kao prirodno proširenje logike sudova:

1. **Generalizacija istinitosti** - od {⊥, ⊤} do [0, 1]
2. **Kolmogorovljevi aksiomi** kao temelj matematičke teorije
3. **Bayesovo zaključivanje** za ažuriranje vjerovanja
4. **Filozofske implikacije** za racionalnost i odlučivanje

Pascal je anticipirao modernu teoriju odlučivanja pokazujući kako matematički pristupiti neizvjesnosti:

> "Razum nas ne može odlučiti između kršćanstva i ateizma, ali morate birati kako ćete živjeti" - Pascal

Vjerojatnost premošćuje jaz između čiste logike i nesigurnog svijeta iskustva. Ona ne daje apsolutnu istinu, ali omogućava racionalno navigiranje kroz neizvjesnost.

Kroz praktično programiranje otkrivamo da je vjerojatnost više od matematičkog alata - ona je jezik za izražavanje stupnjeva vjerovanja, kvantificiranje neizvjesnosti i donošenje racionalnih odluka u svijetu gdje je potpuno znanje nedostižno.

> "Život je umijeće izvlačenja dovoljnih zaključaka iz nedovoljnih premisa" - Samuel Butler

Teorija vjerojatnosti je upravo to umijeće, formalizirano u elegantnom matematičkom okviru.